In [4]:
!pip install kaggle

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
import pandas as pd
import nltk
import re
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords

In [6]:
nltk.download('stopwords')
def RemoveStopWord(clean_tokens):
    stop_words = set(stopwords.words('english'))
    filtered_tokens = [token for token in clean_tokens if token.lower() not in stop_words and len(token) > 1]
    return filtered_tokens

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [7]:
nltk.download('punkt')
def stem_words(word_array):
    stemmer = PorterStemmer()
    stemmed_words = [stemmer.stem(word) for word in word_array]
    return stemmed_words

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [8]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"arianghmgh","key":"33d467c6c0fb3830e865dd6fedeb6075"}'}

In [9]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [10]:
!kaggle datasets download -d tmdb/tmdb-movie-metadata


  0% 0.00/8.89M [00:00<?, ?B/s]
100% 8.89M/8.89M [00:00<00:00, 123MB/s]


In [11]:
!unzip tmdb-movie-metadata.zip


Archive:  tmdb-movie-metadata.zip
  inflating: tmdb_5000_credits.csv   
  inflating: tmdb_5000_movies.csv    


In [12]:
credits = pd.read_csv('tmdb_5000_credits.csv')


In [13]:
credits

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."
...,...,...,...,...
4798,9367,El Mariachi,"[{""cast_id"": 1, ""character"": ""El Mariachi"", ""c...","[{""credit_id"": ""52fe44eec3a36847f80b280b"", ""de..."
4799,72766,Newlyweds,"[{""cast_id"": 1, ""character"": ""Buzzy"", ""credit_...","[{""credit_id"": ""52fe487dc3a368484e0fb013"", ""de..."
4800,231617,"Signed, Sealed, Delivered","[{""cast_id"": 8, ""character"": ""Oliver O\u2019To...","[{""credit_id"": ""52fe4df3c3a36847f8275ecf"", ""de..."
4801,126186,Shanghai Calling,"[{""cast_id"": 3, ""character"": ""Sam"", ""credit_id...","[{""credit_id"": ""52fe4ad9c3a368484e16a36b"", ""de..."


In [14]:
movies = pd.read_csv("tmdb_5000_movies.csv")

In [15]:
movies

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4798,220000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",NaN,9367,"[{""id"": 5616, ""name"": ""united states\u2013mexi...",es,El Mariachi,El Mariachi just wants to play his guitar and ...,14.269792,"[{""name"": ""Columbia Pictures"", ""id"": 5}]","[{""iso_3166_1"": ""MX"", ""name"": ""Mexico""}, {""iso...",1992-09-04,2040920,81.0,"[{""iso_639_1"": ""es"", ""name"": ""Espa\u00f1ol""}]",Released,"He didn't come looking for trouble, but troubl...",El Mariachi,6.6,238
4799,9000,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 10749, ""...",NaN,72766,[],en,Newlyweds,A newlywed couple's honeymoon is upended by th...,0.642552,[],[],2011-12-26,0,85.0,[],Released,A newlywed couple's honeymoon is upended by th...,Newlyweds,5.9,5
4800,0,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 18, ""nam...",http://www.hallmarkchannel.com/signedsealeddel...,231617,"[{""id"": 248, ""name"": ""date""}, {""id"": 699, ""nam...",en,"Signed, Sealed, Delivered","""Signed, Sealed, Delivered"" introduces a dedic...",1.444476,"[{""name"": ""Front Street Pictures"", ""

In [16]:
movies.columns

Index(['budget', 'genres', 'homepage', 'id', 'keywords', 'original_language',
       'original_title', 'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'vote_average',
       'vote_count'],
      dtype='object')

In [36]:
dataframe = movies[['keywords','title','overview','tagline']]
dataframe= dataframe.dropna()
dataframe

,keywords,title,overview,tagline
0,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",Avatar,"In the 22nd century, a paraplegic Marine is di...",Enter the World of Pandora.
1,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","At the end of the world, the adventure begins."
2,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",Spectre,A cryptic message from Bond’s past sends him o...,A Plan No One Escapes
3,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",The Dark Knight Rises,Following the death of District Attorney Harve...,The Legend Ends
4,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",John Carter,"John Carter is a war-weary, former military ca...","Lost in our world, found in another."
...,...,...,...,...
4795,"[{""id"": 10726, ""name"": ""gang""}, {""id"": 33928, ...",Bang,A young woman in L.A. is having a bad day: she...,Sometimes you've got to break the rules
4796,"[{""id"": 1448, ""name"": ""distrust""}, {""id"": 2101...",Primer,Friends/fledgling entrepreneurs invent a devic...,What happens if it actually works?
4798,"[{""id"": 5616, ""name"": ""united states\u2013mexi...",El Mariachi,El Mariachi just wants to play his guitar and ...,"He didn't come looking for trouble, but troubl..."
4799,[],Newlyweds,A newlywed couple's honeymoon is upended by th...,A newlywed couple's honeymoon is upended by th...


In [18]:
def clean_text(text_array):
    output = []
    for i in text_array :
        tokens = nltk.word_tokenize(i)
        a = []
        b = []
        clean_tokens = [re.sub(r'[^a-zA-Z0-9]', '', token) for token in tokens]
        a = RemoveStopWord(clean_tokens)
        for i in a:
            if i !='':
                b.append(i)
        b = stem_words(b)
        tokens_Sport = " ".join(b)
        # if tokens_Sport !='':
        output.append(tokens_Sport)
    return output

In [37]:
Data_after_preproccesing = pd.DataFrame()
Data_after_preproccesing['title'] = clean_text(dataframe['title'])
Data_after_preproccesing['tagline'] = clean_text(dataframe['tagline'])
Data_after_preproccesing['overview'] = clean_text(dataframe['overview'])
Data_after_preproccesing['keywords'] = clean_text(dataframe['keywords'])

In [20]:
Data_after_preproccesing

,title,tagline,overview,keywords
0,avatar,enter world pandora,22nd centuri parapleg marin dispatch moon pand...,id 1463 name cultur clash id 2964 name futur i...
1,pirat caribbean world end,end world adventur begin,captain barbossa long believ dead come back li...,id 270 name ocean id 726 name drug abus id 911...
2,spectr,plan one escap,cryptic messag bond past send trail uncov sini...,id 470 name spi id 818 name base novel id 4289...
3,dark knight rise,legend end,follow death district attorney harvey dent bat...,id 849 name dc comic id 853 name crime fighter...
4,john carter,lost world found anoth,john carter warweari former militari captain i...,id 818 name base novel id 839 name mar id 1456...
...,...,...,...,...
3954,bang,sometim got break rule,young woman la bad day evict audit end produc ...,id 10726 name gang id 33928 name audit id 1727...
3955,primer,happen actual work,friendsfledgl entrepreneur invent devic garag ...,id 1448 name distrust id 2101 name garag id 33...
3956,el mariachi,nt come look troubl troubl came look,el mariachi want play guitar carri famili trad...,id 5616 name unit statesu2013mexico barrier id...
3957,newlyw,newlyw coupl honeymoon upend arriv respect sister,newlyw coupl honeymoon upend arriv respect sister,


In [41]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(Data_after_preproccesing['overview'])
feature_names = vectorizer.get_feature_names_out()
for i, doc in enumerate(Data_after_preproccesing['overview']):
    feature_index = tfidf_matrix[i, :].nonzero()[1]
    tfidf_scores = zip(feature_index, [tfidf_matrix[i, x] for x in feature_index])
    tfidf_scores = sorted(tfidf_scores, key=lambda x: (x[1]), reverse=True)

    print(f"TF-IDF scores for Document {i+1}:")
    for feature_idx, score in tfidf_scores:
        print(f"{feature_names[feature_idx]}: {score}")
    print("\n")

Streaming output truncated to the last 5000 lines.
guil: 0.1860498808030306
beatl: 0.1860498808030306
day: 0.1802850302802956
lennon: 0.17951100668663947
starr: 0.17951100668663947
irrever: 0.17029498123912887
troublemak: 0.17029498123912887
harrison: 0.17029498123912887
1964: 0.17029498123912887
spite: 0.16679121097720023
wildli: 0.16107895579161827
schedul: 0.16107895579161827
element: 0.15868416232098914
grandfath: 0.15272074971285723
press: 0.1510363114132985
fan: 0.13975254152567793
avoid: 0.13528141184939682
arrest: 0.13371323730957685
hard: 0.13296794052444527
televis: 0.13224630799493436
georg: 0.13224630799493436
perform: 0.12956915666381488
reach: 0.1283402349215028
pursu: 0.12210241839143714
wit: 0.11834035540146262
captur: 0.11649140843103266
band: 0.1151134401515601
journey: 0.10657595809180176
john: 0.10459823165240259
night: 0.10356957125375628
use: 0.09902671695036391
life: 0.06731196137998767


TF-IDF scores for Document 3800:
mother: 0.37416396862662354
recreat: 0.328

In [53]:
import math
from collections import Counter
def calculate_tf(document):
    tf_scores = {}
    word_counts = Counter(document.split())
    total_words = len(document.split())
    for word, count in word_counts.items():
        tf_scores[word] = count / total_words
    return tf_scores

def calculate_idf(documents):
    idf_scores = {}
    total_documents = len(documents)
    all_words = set([word for document in documents for word in document.split()])
    for word in all_words:
        document_count = sum([1 for document in documents if word in document.split()])
        idf_scores[word] = math.log(total_documents / (1 + document_count))
    return idf_scores

def calculate_tf_idf(document, documents):
    tf_idf_scores = {}
    tf_scores = calculate_tf(document)
    idf_scores = calculate_idf(documents)
    for word in document.split():
        tf_idf_scores[word] = tf_scores[word] * idf_scores[word]
    return tf_idf_scores

In [60]:
documents = Data_after_preproccesing['overview'].tolist()
document = documents[0]
tf_idf_scores0 = calculate_tf_idf(document, documents)
print(tf_idf_scores0)

{'22nd': 0.43109077412414, 'centuri': 0.2587882512946624, 'parapleg': 0.4744124729091366, 'marin': 0.32745151933641925, 'dispatch': 0.35742483685278714, 'moon': 0.367865717144235, 'pandora': 0.39611478737817607, 'uniqu': 0.32745151933641925, 'mission': 0.21596456311273934, 'becom': 0.13903897197027087, 'torn': 0.3305009045970087, 'follow': 0.20053580824201897, 'order': 0.20932955087346491, 'protect': 0.22928913939155443, 'alien': 0.24543987002603368, 'civil': 0.2955249178510448}


In [62]:
print(tf_idf_scores0)

{'22nd': 0.43109077412414, 'centuri': 0.2587882512946624, 'parapleg': 0.4744124729091366, 'marin': 0.32745151933641925, 'dispatch': 0.35742483685278714, 'moon': 0.367865717144235, 'pandora': 0.39611478737817607, 'uniqu': 0.32745151933641925, 'mission': 0.21596456311273934, 'becom': 0.13903897197027087, 'torn': 0.3305009045970087, 'follow': 0.20053580824201897, 'order': 0.20932955087346491, 'protect': 0.22928913939155443, 'alien': 0.24543987002603368, 'civil': 0.2955249178510448}


In [61]:
documents = Data_after_preproccesing['overview'].tolist()
document = documents[2500]  # Choose the first document for demonstration
tf_idf_scores1 = calculate_tf_idf(document, documents)
print(tf_idf_scores1)

{'bianca': 0.37283526410736434, 'tenth': 0.1941928232010276, 'grader': 0.18641763205368217, 'never': 0.104133241844941, 'gone': 0.1293848428551257, 'date': 0.12223099003817751, 'nt': 0.07962726128121116, 'allow': 0.1293848428551257, 'go': 0.08137154563628064, 'boy': 0.08919297636210255, 'older': 0.13582838402931485, 'sister': 0.10080729339204297, 'kat': 0.7456705282147287, 'get': 0.05974870845961422, 'boyfriend': 0.12100227722529219, 'problem': 0.10944973628402355, 'rub': 0.2051513396363834, 'nearli': 0.133825465862998, 'everyon': 0.10906085148802085, 'wrong': 0.11870071483771633, 'way': 0.07422736432669173, 'guy': 0.10683820018431892, 'eye': 0.11509175828029679, 'joey': 0.36077345052281246, 'eager': 0.15255917344569925, 'fix': 0.14731171359594364, 'patrick': 0.15672540803562515, 'new': 0.05379263427728782, 'kid': 0.09862210159125662, 'town': 0.0829408375539769, 'bitter': 0.15456209161201606, 'enough': 0.12040825271937934}


In [50]:
documents[0]

'22nd centuri parapleg marin dispatch moon pandora uniqu mission becom torn follow order protect alien civil'

In [64]:
from sklearn.metrics.pairwise import cosine_similarity
def calculate_cosine_similarity(vector1, vector2):
    vector1 = list(vector1.values())
    vector2 = list(vector2.values())
    dot_product = sum([x * y for x, y in zip(vector1, vector2)])
    magnitude1 = math.sqrt(sum([x**2 for x in vector1]))
    magnitude2 = math.sqrt(sum([x**2 for x in vector2]))
    similarity = dot_product / (magnitude1 * magnitude2)
    return similarity

In [65]:
calculate_cosine_similarity(tf_idf_scores0,tf_idf_scores1)

0.5819957082048989